In [1]:
import pandas as pd
from utils import *
import os

In [2]:
df = pd.read_csv(f"../data/dataset.csv")
df.head()

,timestamp,id,bikes_available,lat,lon,hour,temperature_celsius,precipitation_mm,humidity_percent,wind_speed_kmh,...,is_weekend,month_sin,month_cos,lag_1,lag_4,lag_8,lag_12,lag_24,roll_mean_4,roll_mean_8
0,2024-12-01 06:15:00,1,1,39.480042,-0.382929,6,11.575,0.0,83.00,5.3,...,1,-2.449294e-16,1.0,1.0,1.0,0.0,2.0,1.0,1.0,0.750
1,2024-12-01 06:30:00,1,1,39.480042,-0.382929,6,11.550,0.0,83.00,5.2,...,1,-2.449294e-16,1.0,1.0,1.0,0.0,2.0,3.0,1.0,0.875
2,2024-12-01 06:45:00,1,1,39.480042,-0.382929,6,11.525,0.0,83.00,5.1,...,1,-2.449294e-16,1.0,1.0,1.0,1.0,0.0,3.0,1.0,1.000
3,2024-12-01 07:00:00,1,1,39.480042,-0.382929,7,11.500,0.0,83.00,5.0,...,1,-2.449294e-16,1.0,1.0,1.0,1.0,0.0,2.0,1.0,1.000
4,2024-12-01 07:15:00,1,0,39.480042,-0.382929,7,11.475,0.0,83.25,5.0,...,1,-2.449294e-16,1.0,1.0,1.0,1.0,0.0,2.0,1.0,1.000


In [3]:
distances = pd.read_csv(f"../data/distance_matrix.csv")
distances.head()

,id,1,10,100,101,102,103,104,106,107,...,90,91,92,93,94,95,96,97,98,99
0,1,0.00,1205.50,4676.66,4982.70,4016.19,3672.19,4170.48,4540.13,5196.34,...,2783.26,4836.61,2969.10,4128.37,3525.53,4303.88,3739.28,4346.31,4011.17,4488.00
1,10,1205.50,0.00,5151.42,5486.41,4490.95,4281.67,4680.40,5050.05,5671.11,...,3065.52,3847.16,3472.81,4632.09,4030.52,4913.36,4243.00,4821.07,3914.37,4991.72
2,100,4676.66,5151.42,0.00,474.85,1040.39,1710.18,1440.57,908.98,532.40,...,1991.78,4915.71,1729.74,665.12,1283.65,1574.82,987.07,462.47,1490.31,277.22
3,101,5256.45,5731.21,592.51,0.00,1620.19,2289.97,1793.81,1262.23,831.69,...,2571.57,5039.05,2309.53,1238.93,1863.44,2065.63,1560.89,1042.26,2064.13,844.83
4,102,4003.14,4486.89,1040.39,1502.24,0.00,1036.66,399.51,671.09,1259.72,...,1318.26,4762.75,1065.21,648.32,597.03,711.95,375.77,710.04,905.43,1007.55


In [4]:
times = pd.read_csv(f"../data/duration_matrix.csv")
times.head()

,id,1,10,100,101,102,103,104,106,107,...,90,91,92,93,94,95,96,97,98,99
0,1,0.00,259.05,971.01,1036.45,855.38,776.18,891.55,954.95,1079.03,...,590.89,999.44,635.07,885.05,757.56,914.30,787.79,928.83,820.36,950.07
1,10,259.05,0.00,1050.18,1114.63,934.54,875.50,977.74,1041.14,1158.19,...,639.35,803.13,713.25,963.22,842.77,1013.63,865.96,1007.99,793.47,1028.25
2,100,971.01,1050.18,0.00,97.50,229.60,370.13,292.20,186.37,115.64,...,432.00,997.37,363.33,155.00,269.58,336.08,199.94,121.45,300.58,79.37
3,101,1092.44,1171.60,129.06,0.00,351.03,491.56,368.86,263.03,180.97,...,553.42,1095.82,484.76,275.24,391.00,442.25,320.18,242.88,420.81,189.52
4,102,836.69,917.65,229.60,319.42,0.00,235.81,106.87,162.65,262.90,...,297.67,977.51,230.80,176.84,132.63,186.38,94.14,187.43,200.06,233.04


In [5]:
times_w = pd.read_csv(f"../data/duration_w_matrix.csv")
times_w.head()

,id,1,10,100,101,102,103,104,106,107,...,90,91,92,93,94,95,96,97,98,99
0,1,0.00,859.66,3104.64,3356.99,2574.54,2260.84,2629.25,3007.21,3412.23,...,1795.13,3406.91,1994.26,2809.91,2236.34,2752.15,2529.91,2776.05,2669.22,3028.65
1,10,859.66,0.00,3080.51,3282.28,2641.91,2420.77,2757.68,3086.80,3399.71,...,1870.19,2634.60,2032.47,2679.90,2355.88,2926.30,2387.95,2763.53,2427.59,2898.64
2,100,3104.64,3080.51,0.00,252.35,745.08,1214.13,1013.60,626.41,363.29,...,1406.72,3325.67,1207.13,408.55,898.21,1093.81,697.04,328.61,883.58,193.85
3,101,3356.99,3282.28,252.35,0.00,997.42,1466.48,1265.95,878.76,530.96,...,1610.38,3324.57,1410.78,602.46,1150.55,1346.15,898.80,580.96,1046.35,383.64
4,102,2574.54,2641.91,745.08,997.42,0.00,534.23,273.32,482.18,865.86,...,876.62,3369.40,677.02,462.63,397.77,376.82,281.62,416.49,630.05,674.64


Preparamos las matrices

In [6]:
# df_distancias o df_tiempos ya cargado
# Asegúrate de que la primera columna se llama "id"
distances.set_index('id', inplace=True)
times.set_index('id', inplace=True)

# Conversión a diccionario de diccionarios
D = distances.to_dict(orient='index') #diccionario de distancias
# Convertimos todas las claves internas a int
D = {
    ext_k: {int(inner_k): v for inner_k, v in inner_dict.items()}
    for ext_k, inner_dict in D.items()
}


T = times.to_dict(orient='index') #diccionario de tiempos
T = {
    ext_k: {int(inner_k): v for inner_k, v in inner_dict.items()}
    for ext_k, inner_dict in T.items()
}

W = times_w.to_dict(orient='index') #diccionario de tiempos
W = {
    ext_k: {int(inner_k): v for inner_k, v in inner_dict.items()}
    for ext_k, inner_dict in T.items()
}
W


{1: {1: 0.0,
  10: 259.05,
  100: 971.01,
  101: 1036.45,
  102: 855.38,
  103: 776.18,
  104: 891.55,
  106: 954.95,
  107: 1079.03,
  108: 1108.45,
  109: 1112.77,
  11: 434.44,
  110: 1106.73,
  111: 1091.73,
  112: 995.64,
  113: 1037.46,
  114: 908.73,
  115: 745.45,
  116: 645.49,
  117: 627.91,
  118: 552.67,
  119: 557.6,
  12: 560.95,
  120: 541.64,
  121: 656.69,
  122: 767.74,
  123: 528.13,
  124: 588.3,
  125: 658.63,
  126: 643.88,
  127: 550.82,
  128: 512.53,
  129: 397.71,
  13: 430.43,
  130: 344.35,
  131: 275.71,
  132: 293.21,
  133: 235.14,
  134: 346.63,
  135: 216.0,
  136: 332.86,
  137: 318.13,
  138: 233.35,
  139: 185.25,
  14: 335.73,
  140: 249.0,
  141: 105.38,
  142: 233.21,
  143: 219.89,
  144: 380.57,
  145: 400.61,
  147: 304.02,
  148: 411.91,
  149: 667.72,
  15: 421.0,
  150: 766.6,
  151: 1015.27,
  152: 625.78,
  153: 126.34,
  154: 1229.57,
  155: 611.41,
  156: 607.57,
  157: 422.89,
  158: 1212.72,
  159: 1077.22,
  16: 472.91,
  160: 999.51,

In [7]:
df_bikes = pd.read_csv(f"../data/bike_stations.csv")
df_bikes.head()

,id,address,total_spaces,geo_shape,lat,lon
0,1,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929
1,16,"Colón, 60",20,"{""coordinates"": [-0.3704333693720587, 39.47009...",39.470092,-0.370433
2,137,Economista Gay - Luis Crumiere,20,"{""coordinates"": [-0.3798963873712934, 39.48903...",39.489033,-0.379896
3,148,C/Xàtiva 30-32,35,"{""coordinates"": [-0.3753782937388751, 39.46714...",39.467145,-0.375378
4,150,Manuel Candela - Rodriguez de Cepeda,25,"{""coordinates"": [-0.35060930364544307, 39.4677...",39.467721,-0.350609


In [8]:
df.columns

Index(['timestamp', 'id', 'bikes_available', 'lat', 'lon', 'hour',
       'temperature_celsius', 'precipitation_mm', 'humidity_percent',
       'wind_speed_kmh', 'weekday', 'is_weekend', 'month_sin', 'month_cos',
       'lag_1', 'lag_4', 'lag_8', 'lag_12', 'lag_24', 'roll_mean_4',
       'roll_mean_8'],
      dtype='object')

Realizamos las predicciones


In [9]:
if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        df = df.copy()
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

is_may_2025 = (
        (df['timestamp'].dt.year  == 2025) &
        (df['timestamp'].dt.month == 5)
)
mayo = df.loc[is_may_2025].copy()

# Load the retrained model
# model_path = os.path.join('model', 'retrained_model.pkl')
model = joblib.load(f"../model/retrained_model.pkl")

# Prepare features and run inference
TARGET = 'bikes_available'
feature_cols = [c for c in mayo.columns if c not in ('timestamp', TARGET)]
X_test = mayo[feature_cols]
mayo['predicted'] = model.predict(X_test)

Calculamos cuantos huecos hay libres en cada momento y estación.

In [10]:
mayo_filtered = mayo[['timestamp','id', 'predicted']]
mayo_merged = mayo_filtered.merge(df_bikes, on='id', how='left')

# Calculamos los espacios disponibles previstos
mayo_merged['available_spaces'] = mayo_merged['total_spaces'] - round(mayo_merged['predicted'])
mayo_merged.head()
    

,timestamp,id,predicted,address,total_spaces,geo_shape,lat,lon,available_spaces
0,2025-05-01 00:00:00,1,2.009686,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,23.0
1,2025-05-01 00:15:00,1,2.980255,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,22.0
2,2025-05-01 00:30:00,1,2.975880,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,22.0
3,2025-05-01 00:45:00,1,2.013927,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,23.0
4,2025-05-01 01:00:00,1,1.077575,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,24.0


In [11]:
df_preds = mayo_merged.set_index(["id", "timestamp"])
P = df_preds["available_spaces"].to_dict()
P

{(1, Timestamp('2025-05-01 00:00:00')): 23.0,
 (1, Timestamp('2025-05-01 00:15:00')): 22.0,
 (1, Timestamp('2025-05-01 00:30:00')): 22.0,
 (1, Timestamp('2025-05-01 00:45:00')): 23.0,
 (1, Timestamp('2025-05-01 01:00:00')): 24.0,
 (1, Timestamp('2025-05-01 01:15:00')): 23.0,
 (1, Timestamp('2025-05-01 01:30:00')): 23.0,
 (1, Timestamp('2025-05-01 01:45:00')): 23.0,
 (1, Timestamp('2025-05-01 02:00:00')): 23.0,
 (1, Timestamp('2025-05-01 02:15:00')): 23.0,
 (1, Timestamp('2025-05-01 02:30:00')): 23.0,
 (1, Timestamp('2025-05-01 02:45:00')): 23.0,
 (1, Timestamp('2025-05-01 03:00:00')): 24.0,
 (1, Timestamp('2025-05-01 03:15:00')): 24.0,
 (1, Timestamp('2025-05-01 03:30:00')): 24.0,
 (1, Timestamp('2025-05-01 03:45:00')): 24.0,
 (1, Timestamp('2025-05-01 04:00:00')): 24.0,
 (1, Timestamp('2025-05-01 04:15:00')): 24.0,
 (1, Timestamp('2025-05-01 04:30:00')): 24.0,
 (1, Timestamp('2025-05-01 04:45:00')): 25.0,
 (1, Timestamp('2025-05-01 05:00:00')): 25.0,
 (1, Timestamp('2025-05-01 05:15:0

In [12]:
from heapq import heappush, heappop
from datetime import datetime, timedelta

def round_to_next_15min(dt):
    # Si ya es múltiplo de 15, no redondeamos
    if dt.minute % 15 == 0 and dt.second == 0 and dt.microsecond == 0:
        return dt.replace(second=0, microsecond=0)
    
    # Si no lo es, redondeamos hacia el siguiente múltiplo de 15
    minute = ((dt.minute // 15) + 1) * 15
    if minute == 60:
        dt += timedelta(hours=1)
        minute = 0
    return dt.replace(minute=minute, second=0, microsecond=0)


def a_star_con_distancia(start, goal, salida_datetime, T, D, P, max_tiempo=30):
    flag = False 
    if start not in T or goal not in T:
        print (f"🚫 Estación {start} o {goal} no está en los datos de rutas (T)")
        return None,{}, flag
    if (start not in D) or (goal not in D):
        print (f"🚫 Estación {start} o {goal} no está en los datos de distancias (D)")
        return None,{}, flag
    # if not any(k[0] == start for k in P.keys()) or not any(k[0] == goal for k in P.keys()):
    #     raise ValueError(f"🚫 No hay predicciones disponibles para start o goal")
    open_set = []
    heappush(open_set, (0, start, salida_datetime, [start]))
    visited = set()
    t_mejor_ini = 0  # Tiempo acumulado al inicio
    info = {}
    
    
    while open_set:
        #sacamos el nodo con manor f (sería como la cota que calculamos)
        f, actual, tiempo_llegada, camino = heappop(open_set)

        # redondeamos el tiempo de llegada al siguiente múltiplo de 15 minutos
        tiempo_pred = round_to_next_15min(tiempo_llegada)

        
        if actual == start:
            try:
                total = mayo_merged.loc[mayo_merged['id'] == start, 'total_spaces'].values[0]
            except IndexError:
                print(f"🚫 Estación {start} no está en el DataFrame.")
                return None,{}, flag

            huecos = P.get((start, tiempo_pred), 0)
            bicis = total - huecos

            if bicis < 1:
                print(f"🚫 No hay bicicletas en {start} a las {tiempo_pred}. Buscando estación alternativa andando...")

                alternativa = None
                mejor_tiempo = float('inf')

                for alterna in W.get(start, {}):  # W = matriz de tiempos andando
                    tiempo_a_pie = round(W[start][alterna] / 60)  # segundos a minutos
                    if tiempo_a_pie > max_tiempo:
                        continue

                    nueva_hora_salida = salida_datetime + timedelta(minutes=tiempo_a_pie)
                    nueva_hora_pred = round_to_next_15min(nueva_hora_salida)

                    try:
                        total_alt = mayo_merged.loc[mayo_merged['id'] == alterna, 'total_spaces'].values[0]
                    except IndexError:
                        continue

                    huecos_alt = P.get((alterna, nueva_hora_pred), 0)
                    bicis_alt = total_alt - huecos_alt

                    if bicis_alt > 0 and tiempo_a_pie < mejor_tiempo:
                        mejor_tiempo = tiempo_a_pie
                        alternativa = (alterna, nueva_hora_salida, nueva_hora_pred)

                if alternativa:
                    alt_id, alt_salida, alt_pred = alternativa
                    print(f"✅ Nueva estación alternativa para salir: {alt_id}")
                    print(f"   🚶 Tiempo andando desde {start}: {mejor_tiempo:.1f} min")
                    print(f"   🕒 Hora de salida estimada desde alternativa: {alt_pred}")
                    t_mejor_ini = mejor_tiempo
                    huecos_disp = P.get((start, tiempo_pred), 0)
                    info[start] = [
                        huecos,
                        bicis,
                        t_mejor_ini,
                        salida_datetime,
                        alt_pred
                    ]

                    heappush(open_set, (0, alt_id, alt_salida, [start, alt_id]))
                    continue
                else:
                    print("❌ No hay estaciones caminando cercanas con bicis disponibles.")
                    return None,{}, flag

            else:
                print(f"🚴 Estación inicial: {start} (total: {total}, bicis: {bicis}, huecos: {huecos})")
        
        start = int(start)
        goal = int(goal)

        t_acum = (tiempo_llegada - salida_datetime).total_seconds() / 60 # tiempo acumulado en minutos

        huecos_disp = P.get((actual, tiempo_pred), 0)
        total = mayo_merged.loc[mayo_merged['id'] == actual, 'total_spaces'].values[0]
        b = total - huecos_disp

        info[actual] = [
            huecos_disp,
            b,
            t_acum,
            tiempo_llegada,
            tiempo_pred
        ]
          
        print(f"\n🚴 Visitando: {actual}")
        print(f"  ⏱ Llegada real: {tiempo_llegada} → redondeado a: {tiempo_pred}")
        print(f"  📦 Huecos disponibles: {P.get((actual, tiempo_pred), 0)}")
        print(f"  🧭 Ruta parcial: {camino}")
        print(f"  ⏳ Tiempo acumulado: {t_acum } min")
        if actual == goal:

            if P.get((goal, tiempo_pred), 0) >= 1:
                return camino, info, flag

            else:
                print(f"❌ Estación destino {goal} sin huecos en {tiempo_pred}. Buscando única alternativa...")

                alternativa = None
                mejor_tiempo = float('inf')

                for alterna in T.get(goal, {}):
                    duracion_extra = T[goal][alterna] / 60  # convertir a minutos
                    if duracion_extra > max_tiempo:
                        continue

                    llegada_alterna = tiempo_llegada + timedelta(minutes=duracion_extra)
                    llegada_alterna_pred = round_to_next_15min(llegada_alterna)

                    if P.get((alterna, llegada_alterna_pred), 0) >= 1:
                        if duracion_extra < mejor_tiempo:
                            mejor_tiempo = duracion_extra
                            alternativa = (alterna, llegada_alterna_pred)

                if alternativa:
                    alterna_id, llegada_pred = alternativa
                    otra = tiempo_llegada + timedelta(seconds=round(mejor_tiempo * 60))
                    t_acum = (otra - salida_datetime).total_seconds() / 60
                    #t_acum += mejor_tiempo
                    
                    
                    print(f"✅ Usamos alternativa final: {alterna_id}")
                    print(f"   ⏱ Tiempo adicional desde {goal}: {mejor_tiempo:.1f} min")
                    print(f"   🧭 Llegada real: {otra} → redondeado a: {llegada_pred}")
                    print(f"   ⌛ Tiempo total desde inicio: {t_acum:.1f} min")
                    huecos_disp = P.get((alterna_id, tiempo_pred), 0)
                    total = mayo_merged.loc[mayo_merged['id'] == alterna_id, 'total_spaces'].values[0]
                    b = total - huecos_disp
                    info[alterna_id] = [
                        huecos_disp,
                        b,
                        t_acum,
                        otra,
                        llegada_pred
                    ]

                    return camino + [alterna_id], info, flag # ✅ finalizamos aquí, no exploramos más



                else:
                    print(f"❌ No se encontró alternativa con hueco para aparcar")
                    return None,{}, flag

                
        # Si no es el destino, añadimos la estación actual al conjunto de visitados
        if actual != start and P.get((actual, tiempo_pred), 0) < 1 and actual != goal:
            continue
        

        visited.add((actual, tiempo_pred)) #añadimos la estacion que acabamos de visitar y el tiempo predicho de llegada
        #print(f"Open set size: {len(open_set)}")  # Debugging line to check open_set size
        #print(f"Visited size: {len(visited)}")  # Debugging line to check visited size
        
        for vecino in T.get(actual, {}): #buscamos los vecinos de la estacion actual
             # Debugging line to check neighbors
            duracion = T[actual][vecino] #nos quedamos con el tiempo desde la estacion actual al vecino
            duracion = round(duracion / 60)  # Convertimos a minutos
            #print(f"Checking neighbor: {vecino}, duration: {duracion} minutes")  # Debugging line to check neighbor and duration
            if duracion > max_tiempo: #si el tiempo de duración es mayor que el máximo permitido, lo saltamos
                continue # saltamos el resto del bucle

            nuevo_tiempo = tiempo_llegada + timedelta(minutes=duracion) # calculamos el nuevo tiempo de llegada para el vecino, usamos timedelta porque la duration matrix está en segundos
            nuevo_tiempo_pred = round_to_next_15min(nuevo_tiempo) #redondeamos de nuevo a lbloque de 15 min
            
            if (vecino, nuevo_tiempo_pred) in visited: #comporbamos que no hemos visitado ya el vecino en ese tiempo predicho
                continue

            if vecino != goal and P.get((vecino, nuevo_tiempo_pred), 0) < 1:
                continue

            # Heurística: distancia desde vecino al destino, que sería la cota optimista que usamos 
            h = D.get(vecino, {}).get(goal, float('inf'))  # km entre vecino y destino
            g = (nuevo_tiempo - salida_datetime).total_seconds() / 60  # tiempo acumulado en minutos
            f_score = g + h  # puedes ponderar h si quieres convertir a tiempo estimado

        


            heappush(open_set, (f_score, vecino, nuevo_tiempo, camino + [vecino]))
        

    return None,0  # no hay ruta válida


In [13]:
mayo_merged.head()

,timestamp,id,predicted,address,total_spaces,geo_shape,lat,lon,available_spaces
0,2025-05-01 00:00:00,1,2.009686,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,23.0
1,2025-05-01 00:15:00,1,2.980255,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,22.0
2,2025-05-01 00:30:00,1,2.975880,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,22.0
3,2025-05-01 00:45:00,1,2.013927,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,23.0
4,2025-05-01 01:00:00,1,1.077575,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,24.0


In [14]:
mayo_merged.loc[mayo_merged['id'] == 268, ['address']].values[0][0]

'Plaza Luis Cano, 5'

In [52]:
import folium
import openrouteservice
from openrouteservice import convert
from folium.plugins import BeautifyIcon

def mapear_ruta(ruta: list, 
                df_estaciones: 'pd.DataFrame', 
                info: dict,
                api_key: str) -> folium.Map:
    """
    Dibuja la ruta en bici usando ORS entre estaciones sobre un mapa interactivo.

    Args:
        ruta (list): Lista de IDs de estaciones por las que pasa la ruta.
        df_estaciones (pd.DataFrame): Debe tener columnas 'id', 'lat', 'lon'.
        api_key (str): Clave de OpenRouteService.

    Returns:
        folium.Map: Mapa interactivo con la ruta ciclista.
    """
    if not ruta or df_estaciones.empty:
        print("⚠️ Ruta vacía o sin datos.")
        return None 
    start = ruta[0] if ruta else None
    last = ruta[-1] if ruta else None
    last_l = ruta[-2] if len(ruta) > 1 else None

    

    client = openrouteservice.Client(key=api_key)
    coords = []

    for est_id in ruta:
        fila = df_estaciones.loc[df_estaciones['id'] == est_id]
        if not fila.empty:
            lat = fila['lat'].values[0]
            lon = fila['lon'].values[0]
            coords.append((lon, lat))  # ORS espera (lon, lat)
        else:
            print(f"⚠️ Estación {est_id} no encontrada en el DataFrame.")

    # Crear mapa centrado
    m = folium.Map(location=(coords[0][1], coords[0][0]), zoom_start=14)

    # Marcadores con tooltip y popup
    #icon_url = "https://cdn-icons-png.flaticon.com/512/684/684908.png"
    for i, (lon, lat) in enumerate(coords):
        huecos, bicis, t_acum, llegada_real, llegada_pred = info[ruta[i]]
        ad = mayo_merged.loc[mayo_merged['id'] == ruta[i], ['address']].values[0][0]
        if ruta[i] == start:    
            c = 'cadetblue' 
            cb = 'teal'       
            label = f"""
                <div style="width:400px;">
                <h2>🚴 Start - Station {ruta[i]}</h2><br>
                <b>Address:</b> {ad}<br>
                🕒 <b>Real arrival hour:</b> {llegada_real.strftime('%Y-%m-%d %H:%M')}<br>
                ⏱ <b>Time:</b> {t_acum:.1f} min<br>
                🚲 <b>Available bikes at {llegada_pred.strftime('%H:%M')}:</b> {bicis}<br>
                🅿️ <b>Available spots at {llegada_pred.strftime('%H:%M')}:</b> {huecos}
                </div>
                """
        elif ruta[i] == last:
            c = 'cadetblue'
            cb = 'teal'  
            label = f"""
                <div style="width:400px;">
                <h2>🚴 End - Station {ruta[i]}</h2><br>
                <b>Address:</b> {ad}<br>
                🕒 <b>Real arrival hour:</b> {llegada_real.strftime('%Y-%m-%d %H:%M')}<br>
                ⏱ <b>Time:</b> {t_acum:.1f} min<br>
                🚲 <b>Available bikes at {llegada_pred.strftime('%H:%M')}:</b> {bicis}<br>
                🅿️ <b>Available spots at {llegada_pred.strftime('%H:%M')}:</b> {huecos}
                </div>
                """
        # elif ruta[i] == last_l:
        #     label = f"🚴 Tramo final - Estación {ruta[i]}"
    
        else:
            c = 'CornflowerBlue'
            cb = 'RoyalBlue'
            label = f"""
                <div style="width:400px;">
                <h2>🚴 Bike Change Station {ruta[i]}</h2><br>
                <b>Address:</b> {ad}<br>
                🕒 <b>Real arrival hour:</b> {llegada_real.strftime('%Y-%m-%d %H:%M')}<br>
                ⏱ <b>Time:</b> {t_acum:.1f} min<br>
                🚲 <b>Available bikes at {llegada_pred.strftime('%H:%M')}:</b> {bicis}<br>
                🅿️ <b>Available spots at {llegada_pred.strftime('%H:%M')}:</b> {huecos}
                </div>
                """
        
        folium.Marker(
            location=(lat, lon),
            popup=label,
            tooltip=f"Station {i+1} / {len(ruta)}",
            icon=BeautifyIcon(
                icon_shape='marker',
                number=ruta[i],
                border_color=cb,
                background_color=c,
                text_color='white'
            )
        ).add_to(m)


    # Añadir rutas reales entre pares consecutivos
        for i in range(len(coords) - 1):
            est_actual = ruta[i]
            est_siguiente = ruta[i + 1]

            # Valor por defecto (color azul)
            color = 'cadetblue'

            # Tramo inicial: sin bicis
            if est_actual == start and info[est_actual][1] == 0:
                color = 'mediumseagreen'

            # Tramo final: sin huecos en penúltima
            if est_actual == last_l and info[est_actual][0] == 0:
                color = 'salmon'

            segmento = client.directions(
                coordinates=[coords[i], coords[i + 1]],
                profile='cycling-regular',
                format='geojson'
            )

            

            folium.GeoJson(
                segmento,
                name=f"Tramo {i+1}",
                style_function=lambda x, col=color: {
                    'color': col,
                    'weight': 5,
                    'opacity': 0.8,
                    'dashArray': '5,5'  # Puedes quitar esta línea si no quieres línea discontinua
                }
            ).add_to(m)

    legend_html = '''
    <div style="
        position: fixed;
        top: 20px;
        right: 20px;
        width: 250px;
        background-color: white;
        border:2px solid grey;
        z-index:9999;
        font-size:11px;
        padding: 10px;
        box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
        ">
        <i class="fa fa-map-marker fa-2x" style="color:cadetblue"></i> Start or Finish Station<br>
        <i class="fa fa-map-marker fa-2x" style="color:CornflowerBlue"></i> Bike Change Station <br>
        <svg width="20" height="10"><line x1="0" y1="5" x2="20" y2="5" stroke="cadetblue" stroke-width="3"/></svg> Path Cycling <br>
        <svg width="20" height="10"><line x1="0" y1="5" x2="20" y2="5" stroke="mediumseagreen" stroke-width="3"/></svg> Path Walking <br>
        <svg width="20" height="10"><line x1="0" y1="5" x2="20" y2="5" stroke="salmon" stroke-width="3"/></svg> Path Cycling (search of bike parking) <br>
        
    </div>
    '''


    m.get_root().html.add_child(folium.Element(legend_html))
       

    return m


## Ejemplos de uso de funcion round_to_nex_15min

In [16]:
ahora = datetime(2025, 6, 19, 11, 13)  # 11:13 AM
#redondear al próximo bloque de 15 minutos
round_to_next_15min(ahora)

datetime.datetime(2025, 6, 19, 11, 15)

## Ejemplos de uso de a_star_con_distancia  y mapa

In [17]:
#uso normal de una estación a otra
salida_datetime = datetime(2025, 5, 1, 8, 0) 
r0,t0,f0 = a_star_con_distancia(90, 162, salida_datetime, T, D, P, max_tiempo=30)
print('_' * 50)
print("Ruta encontrada:", r0)
# print(f"En: {t0:.1f} minutos")
print("Información de estaciones visitadas:")
for k, v in t0.items():
    print(f"  Estación {k}: Huecos: {v[0]}, Bicis disponibles: {v[1]}, Tiempo acumulado: {v[2]:.1f} min, Llegada real: {v[3]}, Predicción: {v[4]}")
        



🚴 Estación inicial: 90 (total: 40, bicis: 3.0, huecos: 37.0)

🚴 Visitando: 90
  ⏱ Llegada real: 2025-05-01 08:00:00 → redondeado a: 2025-05-01 08:00:00
  📦 Huecos disponibles: 37.0
  🧭 Ruta parcial: [90]
  ⏳ Tiempo acumulado: 0.0 min

🚴 Visitando: 162
  ⏱ Llegada real: 2025-05-01 08:12:00 → redondeado a: 2025-05-01 08:15:00
  📦 Huecos disponibles: 3.0
  🧭 Ruta parcial: [90, 162]
  ⏳ Tiempo acumulado: 12.0 min
__________________________________________________
Ruta encontrada: [90, 162]
Información de estaciones visitadas:
  Estación 90: Huecos: 37.0, Bicis disponibles: 3.0, Tiempo acumulado: 0.0 min, Llegada real: 2025-05-01 08:00:00, Predicción: 2025-05-01 08:00:00
  Estación 162: Huecos: 3.0, Bicis disponibles: 16.0, Tiempo acumulado: 12.0 min, Llegada real: 2025-05-01 08:12:00, Predicción: 2025-05-01 08:15:00


In [53]:
ORS_API_KEY= "5b3ce3597851110001cf6248677cc447c3ba4e2fb83501a9f991f890"
mapa = mapear_ruta(r0, mayo_merged, t0, ORS_API_KEY)
#mapa.save("ruta_bici_valenbisi.html")
mapa

In [20]:
# uso de una estación a otra, donde una de las dos estaciones no está disponible
salida_datetime = datetime(2025, 5, 1, 0,30) 
r_, t_, f_ = a_star_con_distancia(268, 146, salida_datetime, T, D, P, max_tiempo=30)
print("Ruta encontrada:", r_)

🚫 Estación 268 o 146 no está en los datos de rutas (T)
Ruta encontrada: None


In [21]:
mapa = mapear_ruta(r_, mayo_merged, t_, ORS_API_KEY)
mapa

⚠️ Ruta vacía o sin datos.


In [22]:
# uso de una estación a otra, donde la estación de destino no tiene huecos disponibles
salida_datetime = datetime(2025, 5, 28, 0,30) 
r1, t1,f1 = a_star_con_distancia(2, 1, salida_datetime, T, D, P, max_tiempo=30)
print('_' * 50)
print("Ruta encontrada:", r1)
for k, v in t1.items():
    print(f"  Estación {k}: Huecos: {v[0]}, Bicis disponibles: {v[1]}, Tiempo acumulado: {v[2]:.1f} min, Llegada real: {v[3]}, Predicción: {v[4]}")

    

🚴 Estación inicial: 2 (total: 15, bicis: 5.0, huecos: 10.0)

🚴 Visitando: 2
  ⏱ Llegada real: 2025-05-28 00:30:00 → redondeado a: 2025-05-28 00:30:00
  📦 Huecos disponibles: 10.0
  🧭 Ruta parcial: [2]
  ⏳ Tiempo acumulado: 0.0 min

🚴 Visitando: 1
  ⏱ Llegada real: 2025-05-28 00:32:00 → redondeado a: 2025-05-28 00:45:00
  📦 Huecos disponibles: 0.0
  🧭 Ruta parcial: [2, 1]
  ⏳ Tiempo acumulado: 2.0 min
❌ Estación destino 1 sin huecos en 2025-05-28 00:45:00. Buscando única alternativa...
✅ Usamos alternativa final: 141
   ⏱ Tiempo adicional desde 1: 1.8 min
   🧭 Llegada real: 2025-05-28 00:33:45 → redondeado a: 2025-05-28 00:45:00
   ⌛ Tiempo total desde inicio: 3.8 min
__________________________________________________
Ruta encontrada: [2, 1, 141]
  Estación 2: Huecos: 10.0, Bicis disponibles: 5.0, Tiempo acumulado: 0.0 min, Llegada real: 2025-05-28 00:30:00, Predicción: 2025-05-28 00:30:00
  Estación 1: Huecos: 0.0, Bicis disponibles: 25.0, Tiempo acumulado: 2.0 min, Llegada real: 2025-

In [54]:
mapa = mapear_ruta(r1, mayo_merged, t1, ORS_API_KEY)
mapa

In [55]:
# uso de una estación a otra, donde la estación de origen no tiene bicicletas disponibles
salida_datetime = datetime(2025, 5, 1, 12,30) 
r2,t2,f2 = a_star_con_distancia(3, 112, salida_datetime, T, D, P, max_tiempo=30)
print('_' * 50)
print("Ruta encontrada:", r2)
for k, v in t2.items():
    print(f"  Estación {k}: Huecos: {v[0]}, Bicis disponibles: {v[1]}, Tiempo acumulado: {v[2]:.1f} min, Llegada real: {v[3]}, Predicción: {v[4]}")
           

🚫 No hay bicicletas en 3 a las 2025-05-01 12:30:00. Buscando estación alternativa andando...
✅ Nueva estación alternativa para salir: 7
   🚶 Tiempo andando desde 3: 2.0 min
   🕒 Hora de salida estimada desde alternativa: 2025-05-01 12:45:00

🚴 Visitando: 7
  ⏱ Llegada real: 2025-05-01 12:32:00 → redondeado a: 2025-05-01 12:45:00
  📦 Huecos disponibles: 17.0
  🧭 Ruta parcial: [3, 7]
  ⏳ Tiempo acumulado: 2.0 min

🚴 Visitando: 112
  ⏱ Llegada real: 2025-05-01 12:51:00 → redondeado a: 2025-05-01 13:00:00
  📦 Huecos disponibles: 40.0
  🧭 Ruta parcial: [3, 7, 112]
  ⏳ Tiempo acumulado: 21.0 min
__________________________________________________
Ruta encontrada: [3, 7, 112]
  Estación 3: Huecos: 20.0, Bicis disponibles: 0.0, Tiempo acumulado: 2.0 min, Llegada real: 2025-05-01 12:30:00, Predicción: 2025-05-01 12:45:00
  Estación 7: Huecos: 17.0, Bicis disponibles: 2.0, Tiempo acumulado: 2.0 min, Llegada real: 2025-05-01 12:32:00, Predicción: 2025-05-01 12:45:00
  Estación 112: Huecos: 40.0, B

In [56]:
mapa = mapear_ruta(r2, mayo_merged, t2, ORS_API_KEY)
mapa

In [28]:
salida_datetime = datetime(2025, 5, 25, 16,44) 
r3,t3,f3 = a_star_con_distancia(268, 162, salida_datetime, T, D, P, max_tiempo=30)
print('_' * 50)
print("Ruta encontrada:", r3)
for k, v in t3.items():
    print(f"  Estación {k}: Huecos: {v[0]}, Bicis disponibles: {v[1]}, Tiempo acumulado: {v[2]:.1f} min, Llegada real: {v[3]}, Predicción: {v[4]}")
           

🚴 Estación inicial: 268 (total: 10, bicis: 9.0, huecos: 1.0)

🚴 Visitando: 268
  ⏱ Llegada real: 2025-05-25 16:44:00 → redondeado a: 2025-05-25 16:45:00
  📦 Huecos disponibles: 1.0
  🧭 Ruta parcial: [268]
  ⏳ Tiempo acumulado: 0.0 min

🚴 Visitando: 150
  ⏱ Llegada real: 2025-05-25 17:14:00 → redondeado a: 2025-05-25 17:15:00
  📦 Huecos disponibles: 20.0
  🧭 Ruta parcial: [268, 150]
  ⏳ Tiempo acumulado: 30.0 min

🚴 Visitando: 162
  ⏱ Llegada real: 2025-05-25 17:22:00 → redondeado a: 2025-05-25 17:30:00
  📦 Huecos disponibles: 3.0
  🧭 Ruta parcial: [268, 150, 162]
  ⏳ Tiempo acumulado: 38.0 min
__________________________________________________
Ruta encontrada: [268, 150, 162]
  Estación 268: Huecos: 1.0, Bicis disponibles: 9.0, Tiempo acumulado: 0.0 min, Llegada real: 2025-05-25 16:44:00, Predicción: 2025-05-25 16:45:00
  Estación 150: Huecos: 20.0, Bicis disponibles: 5.0, Tiempo acumulado: 30.0 min, Llegada real: 2025-05-25 17:14:00, Predicción: 2025-05-25 17:15:00
  Estación 162: Hue

In [29]:
mapa = mapear_ruta(r3, mayo_merged, t3, ORS_API_KEY)
mapa

Estaciones no disponibles

In [30]:
for i in range(1, len(df_bikes['id'].unique()) + 1):
    if i not in df_bikes['id'].unique():
        print(f"ID {i} no está en df_bikes")

ID 105 no está en df_bikes
ID 146 no está en df_bikes
ID 168 no está en df_bikes


Estaciones sin huecos libres

In [31]:
mayo_merged[mayo_merged['available_spaces'] == 0.0]

,timestamp,id,predicted,address,total_spaces,geo_shape,lat,lon,available_spaces
2595,2025-05-28 00:45:00,1,24.532587,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,0.0
3344,2025-05-04 20:00:00,2,14.695023,Salvador Giner - C. Museo,15,"{""coordinates"": [-0.3797483936090024, 39.47988...",39.479889,-0.379748,0.0
3345,2025-05-04 20:15:00,2,14.655168,Salvador Giner - C. Museo,15,"{""coordinates"": [-0.3797483936090024, 39.47988...",39.479889,-0.379748,0.0
3528,2025-05-06 18:00:00,2,14.768633,Salvador Giner - C. Museo,15,"{""coordinates"": [-0.3797483936090024, 39.47988...",39.479889,-0.379748,0.0
3682,2025-05-08 08:30:00,2,14.665487,Salvador Giner - C. Museo,15,"{""coordinates"": [-0.3797483936090024, 39.47988...",39.479889,-0.379748,0.0
...,...,...,...,...,...,...,...,...,...
811990,2025-05-27 05:30:00,276,19.711980,Veles e Vents,20,"{""coordinates"": [-0.32376521655842594, 39.4619...",39.461975,-0.323765,0.0
811991,2025-05-27 05:45:00,276,19.711980,Veles e Vents,20,"{""coordinates"": [-0.32376521655842594, 39.4619...",39.461975,-0.323765,0.0
812298,2025-05-30 10:30:00,276,20.208500,Veles e Vents,20,"{""coordinates"": [-0.32376521655842594, 39.4619...",39.461975,-0.323765,0.0
812300,2025-05-30 11:00:00,276,19.547454,Veles e Vents,20,"{""coordinates"": [-0.32376521655842594, 39.4619...",39.461975,-0.323765,0.0


Comprovación de tiempo entre estaciones

In [32]:
T.get(1, {}).get(2, float('inf')) / 60  # tiempo de 168 a 1

2.829833333333333

Comprobación predicciones para una estación

In [33]:
tiempo_pred = datetime(2025, 5, 1, 15,30) 
P.get((168, tiempo_pred), 0)


0

Comprobación de bicicletas disponibles en estación *start*

In [34]:
start = 1
salida_datetime = datetime(2025, 5, 1, 15,30) 

tiempo_salida_pred = round_to_next_15min(salida_datetime)

try:
    total = mayo_merged.loc[mayo_merged['id'] == start, 'total_spaces'].values[0]
except IndexError:
    print(f"🚫 Estación {start} no está en el DataFrame.")

huecos = P.get((start, tiempo_salida_pred), 0)
bicis = total - huecos

if bicis < 1:
    print(f"🚫 No hay bicicletas disponibles en estación {start} a las {tiempo_salida_pred}")

print(f"🚴 Bicicletas disponibles en estación {start} a las {tiempo_salida_pred}: {bicis}")

🚴 Bicicletas disponibles en estación 1 a las 2025-05-01 15:30:00: 1.0


In [35]:
mayo_merged.loc[mayo_merged['id'] == 3, 'total_spaces'].values[0]

np.int64(20)

In [36]:
mayo_merged.loc[ mayo_merged['timestamp'] ==tiempo_salida_pred , 'available_spaces'].values[0]

np.float64(24.0)

In [37]:
mayo_merged.loc[
    (mayo_merged['id'] == 3) & (mayo_merged['available_spaces'] == 20),
    ['timestamp', 'available_spaces']
]


,timestamp,available_spaces
6002,2025-05-01 12:30:00,20.0
6003,2025-05-01 12:45:00,20.0
6004,2025-05-01 13:00:00,20.0
6005,2025-05-01 13:15:00,20.0
6054,2025-05-02 01:30:00,20.0
...,...,...
8875,2025-05-31 10:45:00,20.0
8876,2025-05-31 11:00:00,20.0
8878,2025-05-31 11:30:00,20.0
8899,2025-05-31 16:45:00,20.0
